In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as plt

In [2]:
from os import listdir
import itertools

In [21]:
#import gensim

In [3]:
%matplotlib inline

In [7]:
path = './'

In [8]:
files = listdir(path)
files

['170327 PURE Data Challenge Powerpoint.pdf',
 'outputs.csv',
 'publications_orgs_counts.csv',
 'JGI-PURE Data Challenge Information Sheet.pdf',
 'org_hierarchy.csv',
 '.ipynb_checkpoints',
 'authors.csv',
 'staff.csv',
 'preprocess.ipynb',
 'readme.txt',
 'org_key.csv']

In [9]:
for s in files:
    if 'csv' in s:
        exec("%s=pd.read_csv(path+s)" % (s[:-4]))
        exec("%s = %s.fillna(0)" % (s[:-4], s[:-4]))
        print s[:-4]
        exec("print %s.shape" % (s[:-4]))

outputs
(34123, 9)
publications_orgs_counts
(318, 8)
org_hierarchy
(447, 2)
authors
(45706, 3)
staff
(8328, 9)
org_key
(448, 5)


In [51]:
mixed = [o not in org_hierarchy.CHILD_ORG_CODE.unique() for o in org_hierarchy.PARENT_ORG_CODE.unique()]

In [58]:
root = org_hierarchy.PARENT_ORG_CODE.unique()[mixed][0]

In [81]:
fw = open(path+'org_vs.gv', 'w')
nodes = [root]
org_lvls={}
lvl=0
org_lvls[root]=(None,lvl)
while len(nodes)!=0:
    node = nodes[0]
    #print node
    tmp = org_hierarchy.loc[org_hierarchy.PARENT_ORG_CODE==node,'CHILD_ORG_CODE'].values
    for c in tmp:
        lvl=org_lvls[node][1]+1
        nodes.append(c)
        org_lvls[c]=(node,lvl)
        fw.writelines(node+' -> '+c+';\n')
    nodes.remove(node)
    
fw.close()

In [82]:
org_lvls

{'ACAD': ('UNIV', 1),
 'ACCO': ('RESS', 4),
 'ACCT': ('ECON', 5),
 'ACFI': ('EFIM', 4),
 'ACHS': ('FSSL', 3),
 'ACRC': ('COSC', 5),
 'ADMO': ('DRAA', 3),
 'ADSI': ('DSTS', 3),
 'AERO': ('QUEN', 4),
 'AFPU': ('DCOM', 4),
 'AGSC': ('SCIF', 4),
 'AHAH': ('VESC', 4),
 'ALSP': ('SMED', 5),
 'ALUM': ('DEVO', 4),
 'AMAR': ('MECH', 5),
 'ANAE': ('CMCC', 5),
 'ANAT': ('CHSE', 4),
 'ANHU': ('FMED', 3),
 'ANTH': ('SSLF', 4),
 'AQPO': ('DARG', 3),
 'ARCH': ('SART', 4),
 'ARTF': ('FOAT', 3),
 'ASUB': ('CASU', 3),
 'ATHF': ('DSEH', 3),
 'AUCC': ('COMP', 5),
 'AUDI': ('ACHS', 4),
 'AVIA': ('PROM', 4),
 'BADO': ('RESS', 4),
 'BBUD': ('ESOP', 4),
 'BBUI': ('HAFM', 4),
 'BBUL': ('ESOP', 4),
 'BCAP': ('DEST', 3),
 'BCCE': ('CHEM', 4),
 'BCOS': ('DEST', 3),
 'BELE': ('BESM', 6),
 'BESM': ('BUAR', 5),
 'BFAC': ('SOFM', 4),
 'BIGC': ('GELY', 4),
 'BIOC': ('FMVS', 3),
 'BIRH': ('ARTF', 4),
 'BISC': ('FSCI', 3),
 'BMEC': ('BESM', 6),
 'BOTA': ('SCIF', 4),
 'BOTG': ('BFAC', 5),
 'BPRO': ('PROM', 4),
 'BPUR': (

In [84]:
def findLvPnode(node,lv,node_dict):
    if node_dict[node][1] <= lv:
        return node
    return findLvPnode(node_dict[node][0],lv,node_dict)

In [87]:
staff.ORGANISATION_CODE = staff.ORGANISATION_CODE.apply(lambda x: findLvPnode(x,3,org_lvls))

In [88]:
staff.head()

,PERSON_ID,PUBLISHED_NAME,FORENAME,SURNAME,ORGANISATION_CODE,TYPE,JOB_TITLE,START_DATE,END_DATE
0,10516,A Putter,Ad,Putter,HUMS,staff,Professor of Medieval English Literature,01/08/2009,31/12/2199
1,20570,MP Crump,Matthew,Crump,CHEM,staff,Professor of NMR and Structural Biology,01/08/2013,31/12/2199
2,16436,JE Weldes,Jutta,Weldes,SPAI,staff,Professor of International Relations,01/08/2010,31/12/2199
3,7125,F Cervantes,Fernando,Cervantes,HUMS,staff,Reader in History,01/08/2009,31/12/2199
4,9179,IG Winson,Ian,Winson,SOCS,staff,Senior Clinical Lecturer in Clinical Sciences,12/07/2013,23/11/2017


In [90]:
orgs = pd.DataFrame()
orgs['ORGANISATION_CODE'] = staff.ORGANISATION_CODE.unique()
orgs.shape

(68, 1)

In [91]:
people = pd.merge(authors,staff, on='PERSON_ID')

In [92]:
orgs['PARENT_CODE']=' '
orgs['ORGANISATION_TYPE']=' '
orgs['FULL_NAME']=' '
for org in orgs.ORGANISATION_CODE.values:
    orgs.loc[orgs.ORGANISATION_CODE==org,'PARENT_CODE'] = org_hierarchy.loc[org_hierarchy.CHILD_ORG_CODE==org,'PARENT_ORG_CODE'].values
    orgs.loc[orgs.ORGANISATION_CODE==org,['ORGANISATION_TYPE','FULL_NAME']] = org_key.loc[org_key.ORGANISATION_CODE==org,['ORGANISATION_TYPE','FULL_NAME']].values

In [93]:
orgs['GRANDPARENT_CODE']=' '
for org in orgs.PARENT_CODE.unique():
    orgs.loc[orgs.PARENT_CODE==org,'GRANDPARENT_CODE']=org_hierarchy.loc[org_hierarchy.CHILD_ORG_CODE==org,'PARENT_ORG_CODE'].values

In [94]:
orgs.head()

,ORGANISATION_CODE,PARENT_CODE,ORGANISATION_TYPE,FULL_NAME,GRANDPARENT_CODE
0,HUMS,FOAT,ACADEMIC SCHOOL,School of Humanities,ACAD
1,CHEM,FSCI,ACADEMIC SCHOOL,School of Chemistry,ACAD
2,SPAI,FSSL,ACADEMIC SCHOOL,"School of Sociology, Politics and Internationa...",ACAD
3,SOCS,FMDY,ACADEMIC SCHOOL,School of Clinical Sciences,ACAD
4,ARTF,FOAT,ACADEMIC OFFICE,Arts Faculty Office,ACAD


In [95]:
people_orgs = pd.merge(people,orgs, on='ORGANISATION_CODE')
people_orgs.head()

,PERSON_ID,PUBLICATION_ID,PUBLISHED_NAME_x,PUBLISHED_NAME_y,FORENAME,SURNAME,ORGANISATION_CODE,TYPE,JOB_TITLE,START_DATE,END_DATE,PARENT_CODE,ORGANISATION_TYPE,FULL_NAME,GRANDPARENT_CODE
0,7761,2449478,AJ Bailey,AJ Bailey,Allen,Bailey,VESC,staff,"Emeritus Professor in Biochemistry, Muscle & C...",01/04/1996,31/12/2199,FMDY,ACADEMIC SCHOOL,School of Veterinary Sciences,ACAD
1,7761,2448863,AJ Bailey,AJ Bailey,Allen,Bailey,VESC,staff,"Emeritus Professor in Biochemistry, Muscle & C...",01/04/1996,31/12/2199,FMDY,ACADEMIC SCHOOL,School of Veterinary Sciences,ACAD
2,25435,2777706,MJ Toscano,Michael J Toscano,Michael,Toscano,VESC,staff,Research Fellow,01/11/2016,30/11/2018,FMDY,ACADEMIC SCHOOL,School of Veterinary Sciences,ACAD
3,25435,2777706,MJ Toscano,M Toscano,Michael,Toscano,VESC,staff,Research Fellow,01/11/2016,30/11/2018,FMDY,ACADEMIC SCHOOL,School of Veterinary Sciences,ACAD
4,25435,2777706,MJ Toscano,MJ Toscano,Michael,Toscano,VESC,staff,Research Fellow,01/11/2016,30/11/2018,FMDY,ACADEMIC SCHOOL,School of Veterinary Sciences,ACAD


In [96]:
DF = pd.merge(people_orgs, outputs, on='PUBLICATION_ID')
DF.head()

,PERSON_ID,PUBLICATION_ID,PUBLISHED_NAME_x,PUBLISHED_NAME_y,FORENAME,SURNAME,ORGANISATION_CODE,TYPE_x,JOB_TITLE,START_DATE,...,FULL_NAME,GRANDPARENT_CODE,TITLE,TYPE_NO,TYPE_y,PUBLICATION_DAY,PUBLICATION_MONTH,PUBLICATION_YEAR,KEYWORDS,ABSTRACT
0,7761,2449478,AJ Bailey,AJ Bailey,Allen,Bailey,VESC,staff,"Emeritus Professor in Biochemistry, Muscle & C...",01/04/1996,...,School of Veterinary Sciences,ACAD,Quantitative determination of collagen cross-l...,5005,contributiontobookanthology/chapter,0.0,0.0,2009,0,0
1,7761,2448863,AJ Bailey,AJ Bailey,Allen,Bailey,VESC,staff,"Emeritus Professor in Biochemistry, Muscle & C...",01/04/1996,...,School of Veterinary Sciences,ACAD,Collagen in its fibrillar state is protected f...,5086,contributiontojournal/article,0.0,3.0,2008,0,0
2,25435,2777706,MJ Toscano,Michael J Toscano,Michael,Toscano,VESC,staff,Research Fellow,01/11/2016,...,School of Veterinary Sciences,ACAD,Dietary long chain omega-3 polyunsaturated fat...,5011,contributiontobookanthology/conference,0.0,0.0,2011,0,0
3,25435,2777706,MJ Toscano,M Toscano,Michael,Toscano,VESC,staff,Research Fellow,01/11/2016,...,School of Veterinary Sciences,ACAD,Dietary long chain omega-3 polyunsaturated fat...,5011,contributiontobookanthology/conference,0.0,0.0,2011,0,0
4,25435,2777706,MJ Toscano,MJ Toscano,Michael,Toscano,VESC,staff,Research Fellow,01/11/2016,...,School of Veterinary Sciences,ACAD,Dietary long chain omega-3 polyunsaturated fat...,5011,contributiontobookanthology/conference,0.0,0.0,2011,0,0


In [101]:
DF.ORGANISATION_CODE.unique()

array(['VESC', 'CHSE', 'EDUC', 'BIOC', 'MVEN', 'BISC', 'PANM', 'SOCS',
       'CHEM', 'MVSF', 'PHPH', 'MATH', 'PSYC', 'ISYS', 'LAWD', 'FSTA',
       'NCCC', 'MEED', 'SPAI', 'HUMS', 'SSCM', 'DPPP', 'ORDS', 'SART',
       'INTL', 'QUEN', 'ENGF', 'GELY', 'GEOG', 'EFIM', 'OPPO', 'SPOL',
       'PERS', 'PHYS', 'PROM', 'MSAD', 'MDYF', 'ATHF', 'FINC', 'IPSS',
       'IMAN', 'STHE', 'SOFM', 'RESS', 'LANG', 'ADSI', 'SCIF', 'BRDC',
       'ORED', 'VICH', 'BCAP', 'MODL', 'SITG', 'CARA', 'UNPA', 'CABI',
       'ADMO', 'WIDE', 'READ', 'ITEL', 'GSEN', 'INOV', 'SAFE', 'NSQI',
       'CRSM', 'INFO', 'ARTF', 'SSLF'], dtype=object)

In [102]:
DF.GRANDPARENT_CODE.unique()

array(['ACAD', 'SUPP', 'UNIV'], dtype=object)

In [103]:
DF.PARENT_CODE.unique()

array(['FMDY', 'FSSL', 'FMVS', 'FENG', 'FSCI', 'DITS', 'DARG', 'CORP',
       'FOAT', 'DSPP', 'DRAA', 'DRES', 'DPER', 'DEST', 'DSEH', 'DFIN',
       'DLIB', 'DSTS', 'DRCO', 'DSTM', 'INST', 'DCMM'], dtype=object)

In [104]:
publication_orgs = {}
for pubid in DF.PUBLICATION_ID.unique():
    wds = DF.loc[DF.PUBLICATION_ID==pubid].ORGANISATION_CODE.unique()    
    if len(wds) > 1:
        yr = DF.loc[DF.PUBLICATION_ID==pubid].PUBLICATION_YEAR.values[0]
        if yr not in publication_orgs.keys():
            publication_orgs[yr] = {}
        for w in itertools.permutations(wds,2):
            if w in publication_orgs[yr].keys():
                publication_orgs[yr][w]+=1
            else: 
                publication_orgs[yr][w]=1
            

In [114]:
orgs_counts = pd.DataFrame(publication_orgs)
orgs_counts=orgs_counts.fillna(0)
orgs_counts.head()

2008  2009  2010  2011  2012  2013
ADMO SSCM   1.0   0.0   0.0   0.0   0.0   0.0
ADSI SOCS   0.0   0.0   0.0   1.0   0.0   0.0
     SSCM   0.0   0.0   0.0   0.0   1.0   0.0
ATHF ISYS   0.0   1.0   1.0   3.0   0.0   0.0
     ORDS   1.0   1.0   3.0   0.0   0.0   0.0

In [115]:
orgs_counts.to_csv(path+'publications_orgs_count.csv')

In [119]:
orgs_counts.loc[orgs_counts.index[0]]

2008    1.0
2009    0.0
2010    0.0
2011    0.0
2012    0.0
2013    0.0
Name: (ADMO, SSCM), dtype: float64

In [122]:
orgs_counts.index[-1]

('WIDE', 'VICH')

In [123]:
fw = open(path+'publications_orgs_count_yrcol.csv', 'w')
for ids in orgs_counts.index:
    for y in orgs_counts.columns:
        v = orgs_counts[y][ids]
        fw.writelines(','.join([str(y),ids[0],ids[1],str(v)])+'\n')
fw.close()

In [124]:
v

0.0